In [1]:
import warnings

from evo.notebooks import ServiceManagerWidget

# Suppress SSL warnings that may occur in JupyterLab
warnings.filterwarnings("ignore", message=".*SSL.*")
warnings.filterwarnings("ignore", message=".*unclosed.*")

# Evo app credentials
client_id = "<your-client-id>"  # Replace with your client ID
redirect_url = "<your-redirect-url>"  # Replace with your redirect URL

client_id = "daves-evo-client"
redirect_url = "http://localhost:32369/auth/callback"

manager = await ServiceManagerWidget.with_auth_code(
    redirect_url=redirect_url,
    client_id=client_id,
).login()

C:\Users\david.knight\Downloads\forks\evo-data-converters\samples\duf-jupyter-conversion\.venv\Lib\site-packages\evo\notebooks\authorizer.py:108: UserWarning: The evo.notebooks.AuthorizationCodeAuthorizer is not secure, and should only ever be used in Jupyter notebooks in a private environment.
  warnings.warn(


ServiceManagerWidget(children=(VBox(children=(HBox(children=(Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR…

Future exception was never retrieved
future: <Future finished exception=ClientOSError(1, '[SSL: APPLICATION_DATA_AFTER_CLOSE_NOTIFY] application data after close notify (_ssl.c:2685)')>
Traceback (most recent call last):
  File "C:\Users\david.knight\AppData\Local\Programs\Python\Python312\Lib\asyncio\sslproto.py", line 648, in _do_shutdown
    self._sslobj.unwrap()
  File "C:\Users\david.knight\AppData\Local\Programs\Python\Python312\Lib\ssl.py", line 921, in unwrap
    return self._sslobj.shutdown()
           ^^^^^^^^^^^^^^^^^^^^^^^
ssl.SSLError: [SSL: APPLICATION_DATA_AFTER_CLOSE_NOTIFY] application data after close notify (_ssl.c:2685)

The above exception was the direct cause of the following exception:

aiohttp.client_exceptions.ClientOSError: [Errno 1] [SSL: APPLICATION_DATA_AFTER_CLOSE_NOTIFY] application data after close notify (_ssl.c:2685)


In [2]:
from helpers.widgets import create_duf_widget

ui = await create_duf_widget(manager)
display(ui)